In [ ]:
from utility_functs import * 
import glob
import os
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import test_functs as test
import openpyxl
from IPython.display import clear_output
import time
import datetime


In [ ]:
#Stored Values on user, runtime and file location
user_name = os.getcwd().split("/")[2].replace("."," ").title()
date_value = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")
files_location = os.path.abspath(os.path.join(os.getcwd(),os.pardir))+"/Drop Files Here" # old method os.getcwd() +'/Drop Files Here'
output_location = os.path.abspath(os.path.join(os.getcwd(),os.pardir))+"/Output Files/"
mapping_file  = pd.read_excel(os.getcwd()+"/Mapping sheet.xlsx")
all_files = glob.glob(os.path.join(files_location, "*")) #I think the argument * will work for now 

#Template information
template = openpyxl.load_workbook("Raw Data Test Output DRAFT.xlsx")
summary_sheet = template["Summary_Metrics"]
tin_roll_up = template['TIN Roll-Up']
npi_roll_up= template['NPI Roll-Up']
plan_roll_up= template['Plan Roll-Up']

#selection Dictionary Constructors
be_dict = {}
for number, entity in enumerate(list(mapping_file.file_type)[2:]):
    be_dict[number] = entity

choice_dict = {}
for number, file in enumerate(all_files):
    choice_dict[number] = file   

#Selection Procedure
print(be_dict)
business_entity_select = input("Select Business Entity: ")
Map = be_dict[int(business_entity_select)]

clear_output()
print("{be} selected".format(be = Map))
time.sleep(2)
clear_output()

for number, file in choice_dict.items():
    print(number,file.split('/')[-1])

#demo_select = input("Select demo file: ")
#elig_select = input("Select Eligibility file: ")
member_select = input("Select Member/Demographic file: ")
prov_select = input("Select Provider Roster file: ")
clear_output()
print("files selected")
time.sleep(2)
clear_output()


#Setting Up the files
list_for_fields = list(list(mapping_file[mapping_file["file_type"] ==Map].values)[0][1:])
list_for_cols = list(list(mapping_file[mapping_file["file_type"]=="stellar_value"].values)[0][1:])
mapping_dict = dict(zip(list_for_cols,(list_for_fields)))

#The individual files themselves may have varying extensions and delimiters so i accounted for that with the function file_ingest
#elig = choice_dict[int(elig_select)]
memb = file_ingest(choice_dict[int(member_select)])
prov = file_ingest(choice_dict[int(prov_select)])
#attr = choice_dict[int(attribution_file_select)]

output_name = choice_dict[int(member_select)].split("/")[-1].split('.')[0]

summary_sheet["B3"].value = Map #DS Name
summary_sheet["B4"].value = date_value #Run Date/Time
summary_sheet["B5"].value = user_name #User

#Field Validation_________________________________________________________________________________________________
test1 = test.ct_memb_rows(pysqldf, memb, mapping_dict)
#summary_sheet["B29"].value,summary_sheet["C29"].value= test1.columns[1] ,test1.iloc[0][1]

test2 = test.ct_memb_membnum_unique(pysqldf,memb, mapping_dict)
summary_sheet["B27"].value,summary_sheet["C27"].value= test2.columns[0] ,test2.iloc[0][0]

test3 = test.ct_memb_membnum_empty(pysqldf,memb, mapping_dict)
summary_sheet["B28"].value,summary_sheet["C28"].value= test3.columns[0] ,test3.iloc[0][0]

test4 = test.ct_memb_membnum_nonunique(pysqldf,memb, mapping_dict)
summary_sheet["B30"].value,summary_sheet["C30"].value= test4.columns[0] ,test4.iloc[0][0]

test5 = test.ct_memb_dob_empty(pysqldf, memb ,mapping_dict)
summary_sheet["B31"].value,summary_sheet["C31"].value= test5.columns[0] ,test5.iloc[0][0]

test6 = test.ct_memb_fname_empty(pysqldf,memb ,mapping_dict)
summary_sheet["B34"].value,summary_sheet["C34"].value= test6.columns[0] ,test6.iloc[0][0]

test7 = test.ct_memb_fname_length(pysqldf,memb ,mapping_dict)
summary_sheet["B35"].value,summary_sheet["C35"].value= test7.columns[0] ,test7.iloc[0][0]

test8 = test.ct_memb_lname_empty(pysqldf,memb ,mapping_dict)
summary_sheet["B36"].value,summary_sheet["C36"].value= test8.columns[0] ,test8.iloc[0][0]

test9 = test.ct_memb_lname_length(pysqldf,memb ,mapping_dict)
summary_sheet["B37"].value,summary_sheet["C37"].value= test9.columns[0] ,test9.iloc[0][0]

test10 = test.ct_memb_tin_empty(pysqldf,memb ,mapping_dict)
summary_sheet["B37"].value,summary_sheet["C37"].value= test10.columns[0] ,test10.iloc[0][0]

#Provider File Fields___________________________________________________________________________
test11 = test.ct_prov_npi_empty(pysqldf,prov,mapping_dict)
summary_sheet["B72"].value,summary_sheet["C72"].value= test11.columns[0] ,test11.iloc[0][0]

#test12 = test.ct_prov_npi_valid(pysqldf,prov,mapping_dict)
#summary_sheet["B70"].value,summary_sheet["C70"].value= test12.columns[0] ,test12.iloc[0][0]

#Attribution Validation___________________________________________________________________________
test13 = test.ct_memb_attr_npi_unique(pysqldf,prov,mapping_dict)
summary_sheet["B56"].value,summary_sheet["C56"].value= test13.columns[0] ,test13.iloc[0][0]

test14 = test.ct_prov_tin_unique(pysqldf,prov, mapping_dict)
summary_sheet["B74"].value,summary_sheet["C74"].value= test14.columns[0] ,test14.iloc[0][0]

test19 = test.ct_memb_attr_npi_lookup(pysqldf,memb,prov,mapping_dict)
summary_sheet["B83"].value,summary_sheet["C83"].value= test19.columns[0] ,test19.iloc[0][0]

test20 = test.ct_memb_prov_attr_npi_multi_tins(pysqldf,memb,prov,mapping_dict)
summary_sheet["B74"].value,summary_sheet["C74"].value= test20.columns[0] ,test20.iloc[0][0]


#@1
test21 = test.rpt_memb_demographics(pysqldf,memb ,mapping_dict)

test22 = test.rpt_memb_sex_valuelist(pysqldf,memb ,mapping_dict)
#test13 = test.ct_memb_attr_npi_invalid(pysqldf,memb ,mapping_dict)
#@1


#probably errors out because no DOD field - Yep it failed
#test17 = test.date_of_death_format_check(pysqldf, memb,mapping_dict)

#template.create_sheet("Count of Member Numbers") 
#test1.to_excel(template, sheet_name = "Count of Member Numbers")
template.save(output_location+ output_name+"_summary.xlsx") #-- active export method
print("Exporting file {name} to your computer. Thank you for using Raw Data Testing, {user}".format(name = output_name +"_summary.xlsx", user = user_name))

In [ ]:
writer2 = pd.ExcelWriter("test_output.xlsx", engine='xlsxwriter')
workbook2=writer2.book
worksheet2=workbook2.add_worksheet('Member Number Metrics')
writer2.sheets['Demographic Metrics'] = worksheet
#test2..to_excel
#test2.columns[0] 
pd.DataFrame.to_excel(test2,writer2,sheet_name='Demographic Metrics',startrow=0 , startcol=2)
writer2.save()
